In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/make-data-count-finding-data-references/sample_submission.csv
/kaggle/input/make-data-count-finding-data-references/train_labels.csv
/kaggle/input/make-data-count-finding-data-references/test/XML/10.1002_ece3.5260.xml
/kaggle/input/make-data-count-finding-data-references/test/XML/10.1002_chem.201902131.xml
/kaggle/input/make-data-count-finding-data-references/test/XML/10.1002_ece3.3985.xml
/kaggle/input/make-data-count-finding-data-references/test/XML/10.1002_ejoc.202000916.xml
/kaggle/input/make-data-count-finding-data-references/test/XML/10.1002_ece3.6144.xml
/kaggle/input/make-data-count-finding-data-references/test/XML/10.1002_2017jc013030.xml
/kaggle/input/make-data-count-finding-data-references/test/XML/10.1002_anie.201916483.xml
/kaggle/input/make-data-count-finding-data-references/test/XML/10.1002_chem.202001412.xml
/kaggle/input/make-data-count-finding-data-references/test/XML/10.1002_chem.202003167.xml
/kaggle/input/make-data-count-finding-data-references/test/X

/kaggle/input/make-data-count-finding-data-references/train/XML/10.1590_1678-4685-gmb-2018-0055.xml
/kaggle/input/make-data-count-finding-data-references/train/XML/10.1021_jacs.2c06519.xml
/kaggle/input/make-data-count-finding-data-references/train/XML/10.1107_s2056989015019891.xml
/kaggle/input/make-data-count-finding-data-references/train/XML/10.1186_s12881-019-0773-3.xml
/kaggle/input/make-data-count-finding-data-references/train/XML/10.3762_bjoc.8.42.xml
/kaggle/input/make-data-count-finding-data-references/train/XML/10.1038_s41597-023-02280-2.xml
/kaggle/input/make-data-count-finding-data-references/train/XML/10.1093_bib_bbab395.xml
/kaggle/input/make-data-count-finding-data-references/train/XML/10.1186_s12918-018-0564-z.xml
/kaggle/input/make-data-count-finding-data-references/train/XML/10.1017_s0007123423000601.xml
/kaggle/input/make-data-count-finding-data-references/train/XML/10.3133_fs20233046.xml
/kaggle/input/make-data-count-finding-data-references/train/XML/10.3133_ofr2023

/kaggle/input/make-data-count-finding-data-references/train/PDF/10.1371_journal.pone.0191086.pdf
/kaggle/input/make-data-count-finding-data-references/train/PDF/10.1140_epjc_s10052-018-6033-4.pdf
/kaggle/input/make-data-count-finding-data-references/train/PDF/10.1098_rsos.160417.pdf
/kaggle/input/make-data-count-finding-data-references/train/PDF/10.1002_cssc.202201821.pdf
/kaggle/input/make-data-count-finding-data-references/train/PDF/10.1186_s13046-018-0843-y.pdf
/kaggle/input/make-data-count-finding-data-references/train/PDF/10.1371_journal.pone.0284951.pdf
/kaggle/input/make-data-count-finding-data-references/train/PDF/10.1093_nar_gkp1049.pdf
/kaggle/input/make-data-count-finding-data-references/train/PDF/10.1107_s2059798322005691.pdf
/kaggle/input/make-data-count-finding-data-references/train/PDF/10.1038_hdy.2015.99.pdf
/kaggle/input/make-data-count-finding-data-references/train/PDF/10.3897_jhr.96.111550.pdf
/kaggle/input/make-data-count-finding-data-references/train/PDF/10.1590_19

In [2]:
!pip install lxml
!pip install --quiet PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/24.1 MB ? eta -:--:--

   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/24.1 MB 10.7 MB/s eta 0:00:03

   ━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━ 10.5/24.1 MB 138.7 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━ 18.3/24.1 MB 215.9 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 24.1/24.1 MB 228.8 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 79.5 MB/s eta 0:00:00


### Parsing PDF and XML files

In [3]:
import os
from lxml import etree
import fitz  # PyMuPDF
#fitz is a high-performance library for working with PDF files enabling the extraction og text, imgs etc

def extract_text(path):
    """
    Parse XML and PDF files and extract text content from title, abstract, and body sections.
    Args:
        path (str): path to the XML or PDF file
    Returns:
        article_id (str): article ID from filename
        sections (list of dict): each dict contains:
            - section: section title
            - text: full paragraph text of that section
    """
    sections = [] #empty list where the results will be saved
    article_id = os.path.basename(path).rsplit('.', 1)[0]  # remove extension and save article Id
    
    if path.lower().endswith('.xml'): #check if doc is XML or not
        # XML parsing
        parser = etree.XMLParser(recover=True) #parser instance; recover=True- allow parsin even if it encounters malformed XML structure
        with open(path, 'rb') as f:
            tree = etree.parse(f, parser)
            root = tree.getroot()

        ns = {'ns': root.nsmap.get(None)} if None in root.nsmap else {}

        # 1. title
        title_el = root.find(".//article-title", namespaces=ns)
        if title_el is not None and title_el.text:
            sections.append({'section': 'Title', 'text': title_el.text.strip()})

        # 2. abstract
        for abs_el in root.findall(".//abstract", namespaces=ns):
            abstract_text = " ".join(abs_el.itertext()).strip()
            if abstract_text:
                sections.append({'section': 'Abstract', 'text': abstract_text})

        # 3. body sections
        for sec_el in root.findall(".//sec", namespaces=ns):
            sec_title_el = sec_el.find("title", namespaces=ns)
            sec_title = sec_title_el.text.strip() if sec_title_el is not None and sec_title_el.text else "No Title"
            
            para = [p.text for p in sec_el.findall(".//p", namespaces=ns) if p.text]
            section_text = " ".join(para).strip()

            if section_text:
                sections.append({'section': sec_title, 'text': section_text})

    elif path.lower().endswith('.pdf'):
        # PDF parsing
        try:
            text = ""
            with fitz.open(path) as doc:
                for page in doc:
                    text += page.get_text("text") + "\n"
            if text.strip():
                sections.append({'section': 'FullText', 'text': text.strip()})
        except Exception as e:
            print(f"Error reading PDF {path}: {e}")

    else:
        print(f"Unsupported file type: {path}")

    return article_id, sections

In [4]:
# Path to one training article
xml_path = "/kaggle/input/make-data-count-finding-data-references/train/XML/10.1186_s13059-019-1918-6.xml"

# Extract sections
article_id, sections = extract_text(xml_path)

print(f"Article ID: {article_id}")
for sec in sections[:3]:  # show first 3 sections
    print("\n--- Section:", sec.get('section', 'Unknown'), "---")
    print(sec.get('text', '[No text available]')[:500])

Article ID: 10.1186_s13059-019-1918-6

--- Section: Title ---
Enteric infection induces Lark-mediated intron retention at the 5′ end of

--- Section: Abstract ---
Background RNA splicing is a key post-transcriptional mechanism that generates protein diversity and contributes to the fine-tuning of gene expression, which may facilitate adaptation to environmental challenges. Here, we employ a systems approach to study alternative splicing changes upon enteric infection in females from classical  Drosophila melanogaster  strains as well as 38 inbred lines. Results We find that infection leads to extensive differences in isoform ratios, which results in a mor

--- Section: Background ---
RNA splicing is a key post-transcriptional mechanism that generates protein diversity and contributes to the fine-tuning of gene expression, which may facilitate adaptation to environmental challenges. Here, we employ a systems approach to study alternative splicing changes upon enteric infection in female

### Load train_label.csv : ground truth

In [5]:
labels_df=pd.read_csv("/kaggle/input/make-data-count-finding-data-references/train_labels.csv")
print(labels_df.head())

               article_id                                 dataset_id     type
0    10.1002_2017jc013030             https://doi.org/10.17882/49388  Primary
1  10.1002_anie.201916483  https://doi.org/10.5517/ccdc.csd.cc1npvt0  Missing
2  10.1002_anie.202005531  https://doi.org/10.5517/ccdc.csd.cc24wxqp  Missing
3  10.1002_anie.202007717  https://doi.org/10.5517/ccdc.csd.cc24rrb0  Missing
4  10.1002_chem.201902131  https://doi.org/10.5517/ccdc.csd.cc221dk3  Missing


In [6]:
!pip install --quiet pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/42.8 kB ? eta -:--:--

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.3 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.9 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.4 MB/s eta 0:00:00


   ━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/5.6 MB 36.5 MB/s eta 0:00:01

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 58.1 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/2.8 MB ? eta -:--:--

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 47.7 MB/s eta 0:00:00


### parallel processing to avoid delay and processing botttleneck issues

In [7]:
from joblib import Parallel, delayed
import os
import pandas as pd
import pdfplumber
from tqdm import tqdm

# base paths
xml_dir = "/kaggle/input/make-data-count-finding-data-references/train/XML"
pdf_dir = "/kaggle/input/make-data-count-finding-data-references/train/PDF"

# function to process one row (article-dataset pair)
def process_row(row):
    article_id = row['article_id']
    dataset_id = str(row['dataset_id']).strip()
    label_type = row['type']

    if dataset_id == 'nan' or pd.isna(dataset_id):
        return None

    # ---- XML ----
    xml_path = os.path.join(xml_dir, f"{article_id}.xml")
    try:
        if os.path.exists(xml_path):
            _, sections = extract_text(xml_path)
            for sec in sections:
                text = sec.get('text', '')
                if dataset_id.lower() in text.lower():
                    return {
                        "article_id": article_id,
                        "dataset_id": dataset_id,
                        "text": text,
                        "type": label_type,
                        "source": "XML"
                    }
    except Exception as e:
        return None  # optionally log e

    # ---- PDF ----
    pdf_path = os.path.join(pdf_dir, f"{article_id}.pdf")
    try:
        if os.path.exists(pdf_path):
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    text = page.extract_text() or ""
                    if dataset_id.lower() in text.lower():
                        return {
                            "article_id": article_id,
                            "dataset_id": dataset_id,
                            "text": text,
                            "type": label_type,
                            "source": "PDF"
                        }
    except Exception as e:
        return None  # optionally log e

    return None  # no match found

# run in parallel
results = Parallel(n_jobs=-1)(
    delayed(process_row)(row) for _, row in tqdm(labels_df.iterrows(), total=len(labels_df))
)

# keep only successful matches
training_data = [r for r in results if r is not None]

# convert to DataFrame
training_df = pd.DataFrame(training_data)
print(training_df.head())

  0%|          | 0/1066 [00:00<?, ?it/s]

  1%|          | 8/1066 [00:02<06:09,  2.86it/s]

  1%|          | 12/1066 [00:06<10:49,  1.62it/s]

  2%|▏         | 16/1066 [00:10<12:38,  1.38it/s]

  2%|▏         | 20/1066 [00:16<17:58,  1.03s/it]

...

  6%|▌         | 64/1066 [01:36<29:27,  1.76s/it]

  6%|▋         | 68/1066 [01:57<46:12,  2.78s/it]

  7%|▋         | 72/1066 [02:14<54:02,  3.26s/it]

  7%|▋         | 76/1066 [02:21<46:07,  2.80s/it]

  8%|▊         | 80/1066 [02:31<44:39,  2.72s/it]

In [ ]:
df = pd.DataFrame(training_data)
print(df.head())

In [ ]:
print(df['type'].value_counts())

### encode labels

In [ ]:
df = df[df['type'].isin(['Primary', 'Secondary'])]  # remove missing or noisy labels

label_map = {'Primary': 1, 'Secondary': 0}
df['label'] = df['type'].map(label_map)

In [ ]:
from sklearn.utils import resample

# Separate classes
primary = df[df['type'] == 'Primary']
secondary = df[df['type'] == 'Secondary']

# Downsample secondary to match primary count
secondary_down = resample(secondary, replace=False, n_samples=len(primary), random_state=42)

# Combine
balanced_df = pd.concat([primary, secondary_down])

In [ ]:
print(balanced_df.columns)
print(balanced_df.head())

### split dataset

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    balanced_df['text'],balanced_df['label'], test_size=0.2, random_state=42, stratify=balanced_df['label']
)

### TF-IDF + logistic regression

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix

pipeline = make_pipeline(
    TfidfVectorizer(max_features=5000, ngram_range=(1,2)),
    LogisticRegression(max_iter=1000, class_weight='balanced')
)

pipeline.fit(X_train, y_train)

# Evaluate
y_pred = pipeline.predict(X_val)
print(classification_report(y_val, y_pred, target_names=["Secondary", "Primary"]))

In [ ]:
import re
def extract_ids(text):
    """
    extract potential dataset identifiers from a text section
    """
    ids =set()

#DOI based datasets
    doi= r"(10\.\d{4,9}/[-._;()/:A-Z0-9]+)"
    ids.update(re.findall(doi, text, flags=re.I))

#accession-style datasets
    acc=r"\b(GSE\d+|E-MEXP-\d+|PRJ\w+\d+|PDB\s?\w+)\b"
    ids.update(re.findall(acc, text, flags=re.I))

    return list(ids)

### Prediction

In [ ]:
test_xml_dir = "/kaggle/input/make-data-count-finding-data-references/test/XML"
files = [f for f in os.listdir(test_xml_dir) if f.endswith(".xml")]

print("Total XML files in test folder:", len(files))

predictions = []
seen = set()  # avoid duplicates

for file in tqdm(files, desc="Processing test XMLs"):
    article_id = file.replace(".xml", "")
    xml_path = os.path.join(test_xml_dir, file)

    try:
        # Extract sections (assuming you already defined extract_text)
        _, sections = extract_text(xml_path)

        article_has_prediction = False  # track if we captured anything

        for sec in sections:
            text = sec.get('text', '')
            if not text:
                continue

            # 1. Predict citation type
            pred_type = pipeline.predict([text])[0]

            # 2. Extract dataset IDs
            dataset_ids = extract_ids(text)

            if dataset_ids:
                for dataset_id in dataset_ids:
                    dataset_id = normalize_dataset_id(dataset_id)
                    if not dataset_id:
                        continue

                    key = (article_id, dataset_id, pred_type)
                    if key not in seen:
                        seen.add(key)
                        predictions.append({
                            "article_id": article_id,
                            "dataset_id": dataset_id,
                            "type": pred_type
                        })
                        article_has_prediction = True

        # Fallback: ensure at least one row per article
        if not article_has_prediction:
            pred_type = pipeline.predict([" ".join([sec.get('text', '') for sec in sections])])[0]
            predictions.append({
                "article_id": article_id,
                "dataset_id": "N/A",
                "type": pred_type
            })

    except Exception as e:
        print(f"Failed on {article_id}: {e}")


In [ ]:
submission = pd.DataFrame(predictions)
submission.to_csv("submission.csv", index=False)
print("Final submission shape:", submission.shape)
submission.head()